In [1]:
from dataset import AVADataset, AVADataLoader

In [2]:
batch_size = 1

train_dataset = AVADataset(mode="train")
val_dataset = AVADataset(mode="val")
test_dataset = AVADataset(mode="test")

train_loader = AVADataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = AVADataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = AVADataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Creating dataset train


Processing cWYJHb25EVs:   0%|          | 0/15 [00:00<?, ?it/s]

Dataframes already exist, skipping processing
Dataframes already exist, skipping processing


Processing cWYJHb25EVs: 100%|██████████| 15/15 [00:00<00:00, 68.10it/s]

Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing


...

Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing


Processing Sntyb4omSfU: 100%|██████████| 15/15 [00:00<00:00, 206.20it/s]


Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing


Processing tj-VmrMYtUI: 100%|██████████| 12/12 [00:00<00:00, 250.73it/s]

Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing
Dataframes already exist, skipping processing


In [3]:
len(train_dataset), len(val_dataset), len(test_dataset)

(838, 765, 476)

In [4]:
for i, batch in enumerate(train_loader):
    break

audio, visual, targets = batch

In [ ]:
import torch
from torch import nn
from VisualSAD import AudioEncoder, VisualEncoder, AudioVisualFusion, TemporalModel, ContrastiveLoss

class VisualSAD(nn.Module):
    """
    Main model class for VisualSAD.
    Input shape is (batch_size, max_n_speakers, C, T, H, W) for visual data
    and (batch_size, 4*T, C_mel) for audio data.
    """

    def __init__(self,
                 C, T, H, W, C_mel,
                 embedding_dim_audio,
                 embedding_dim_visual,
                 embedding_dim,
                 target_dim,
                 lmbda=0.3):
        super(VisualSAD, self).__init__()

        self.C = C  # Number of channels in the visual data
        self.T = T  # Number of time frames in the visual data
        self.H = H  # Height of the visual data
        self.W = W  # Width of the visual data
        self.C_mel = C_mel  # Number of mel bands in the audio data
        self.embedding_dim_audio = embedding_dim_audio
        self.embedding_dim_visual = embedding_dim_visual
        self.embedding_dim = embedding_dim  # Dimension of the final embedding
        self.target_dim = target_dim  # Number of classes for classification

        # Lambda parameter to balance the influence of the contrastive loss
        self.lmbda = lmbda

        input_shape_audio = (4 * T, C_mel)
        input_shape_visual = (C, T, H, W)

        self.audio_encoder = AudioEncoder(
            input_shape=input_shape_audio,
            output_dim=embedding_dim_audio)
        self.visual_encoder = VisualEncoder(
            input_shape=input_shape_visual,
            output_dim=embedding_dim_visual)

        self.fusion = AudioVisualFusion(
            embedding_dim_audio=embedding_dim_audio,
            embedding_dim_visual=embedding_dim_visual,
            output_dim=embedding_dim)

        self.temporal_model = TemporalModel(
            input_dim=embedding_dim,
            output_dim=target_dim)

        # Contrastive loss function
        self.contrastive_loss = ContrastiveLoss()
        # Final loss function for classification of speakers
        self.classification_loss = nn.CrossEntropyLoss()

    def forward(self, audio, visual, targets):
        """Forward pass through the model."""
        batch_size = audio.size(0)
        n_speakers = visual.size(1)

        audio_embedding = self.audio_encoder(audio)

        # Collapse the first two dimensions of the visual input
        # (batch_size * n_speakers, C, T, H, W)
        visual = visual.view(-1, self.C, self.T, self.H, self.W)
        visual_embedding = self.visual_encoder(visual)

        # print(
        #    f"Audio embedding shape after encoder: {audio_embedding.shape}")
        # print(
        #    f"Visual embedding shape after encoder: {visual_embedding.shape}")

        # Compute the dot product of the audio and visual embeddings
        # Broadcast the audio embedding to match the visual embedding shape
        # Shape is (batch_size, n_speakers, T, embedding_dim_audio)
        audio_embedding = audio_embedding.unsqueeze(1)
        audio_embedding = audio_embedding.expand(
            -1, n_speakers, -1, -1)
        # Collapse the first two dimensions of the audio embedding
        # (batch_size * n_speakers, T, embedding_dim_audio)
        audio_embedding = audio_embedding.contiguous().view(
            -1, self.T, self.embedding_dim_audio)

        print(
           f"Audio embedding shape after expanding: {audio_embedding.shape}")
        print(
           f"Visual embedding shape after reshaping: {visual_embedding.shape}")
        
        # Select only the active frames i.e. the frames where at least one
        # speaker is speaking (targets[..., t, :])
        # and compute the contrastive loss using the
        # corresponding embeddings
        targets = targets.view(batch_size*n_speakers, self.T, -1)
        
        if self.training:
            act_idx = targets.sum(dim=-1) > 0
            if act_idx.sum() > 0:
                act_embedding_audio = audio_embedding[act_idx]
                act_embedding_visual = visual_embedding[act_idx]
                print(
                    f"Active frames shape: {act_idx.shape}")
                print(
                    f"Active audio hape: {act_embedding_audio.shape}")
                print(
                    f"Active visual shape: {act_embedding_visual.shape}")
                # Shape is (batch_size * n_speakers, T_act, embedding_dim_audio)
                # Compute the dot product along the 2 last dimensions
                s = torch.matmul(
                    act_embedding_audio, act_embedding_visual.transpose(1, 2))
                # Compute the contrastive loss
                loss_c = self.contrastive_loss(s)
            else:
                # If no active frames are present, set the loss to 0
                loss_c = torch.tensor(0.0).to(audio.device)

        # Apply the fusion module to get the final embedding
        # Shape is (batch_size, T, embedding_dim)
        x = self.fusion(audio_embedding, visual_embedding)

        # Apply the temporal model to get the predictions
        x = self.temporal_model(x)
        x = x.view(batch_size*n_speakers, self.T, x.size(-1))
        # print(f"Shape after temporal model: {x.shape}")

        if self.training:
            # Compute the classification loss
            loss_cls = self.classification_loss(x.view(-1, x.size(-1)), targets.view(-1))
            total_loss = loss_cls + self.lmbda * loss_c
       
        # print(f"Targets shape: {targets.shape}")
        targets = targets.view(batch_size*n_speakers, self.T, -1)
        loss_cls = self.classification_loss(x, targets)

        # Reshape the output to (batch_size, n_speakers, output_dim)
        if self.training:    
            return total_loss, x.view(-1, x.size(1), x.size(2)), loss_cls, loss_c
        return x.view(-1, x.size(1), x.size(2))
    
sad = VisualSAD(
    C=1,
    T=32,
    H=112,
    W=112,
    C_mel=64,
    embedding_dim_audio=128,
    embedding_dim_visual=128,
    embedding_dim=128,
    target_dim=1
)

In [9]:
# Get the number of trainable parameters in the model
def get_n_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {get_n_params(sad)}")

Number of trainable parameters: 461648812


In [7]:
sad(audio, visual, targets)

Audio embedding shape after expanding: torch.Size([1, 32, 128])
Visual embedding shape after reshaping: torch.Size([1, 32, 128])
Active frames shape: torch.Size([1, 32])
Active frames shape: torch.Size([7, 128])
Active frames shape: torch.Size([7, 128])


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)